Enter Path to the folder you wish to process:

In [1]:
mainFolder = "/data/Sandra/2020/"

In [2]:
## importing stuff
import os
import pickle
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
import javabridge
from bioformats import JARS as bfJARS
javabridge.start_vm(class_path=bfJARS, max_heap_size="6G")
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 2
# %aimport islets

from IPython.display import display

import pandas as pd

from islets.Recording import Recording
from islets.Regions import Regions
from islets.numeric import power_spectrum, rebin
from islets.utils import show_movie, saveRois

from islets.Regions import getPeak2BoundaryDF, getGraph_of_ROIs_to_Merge, mergeBasedOnGraph

from caiman import movie as cmovie

from matplotlib.colors import LogNorm

def get_filterSizes(px):
    base = int(np.ceil(5/px))
    wider = int(np.ceil(base*1.1))
    if wider==base: wider += 1
    toComb = int(np.ceil(base*1.3))
    if toComb <= wider: toComb += 1
    return [(base,), (wider,), (base,wider), (base,toComb)]

def parse_leica(rec):
    from pandas import Timedelta
    toDrop = [i for i,row in rec.metadata.iterrows() if "Series" not in row["Name"]]
    rec.metadata.drop(index=toDrop, inplace=True)
    if len(rec.metadata)>1:
        rec.calc_gaps()
        ff = np.any(
                [rec.metadata["gap"]>5]+
                [rec.metadata["gap"]<-5]+
                [rec.metadata[c].diff().abs()>0 for c in ["pxSize", "SizeX", "SizeY"]],
            axis=0)
        sers = np.split(rec.metadata.Name.values, np.where(ff)[0])
    else:
        sers = [rec.metadata.Name.values]
    
    outSer = []
    for serlist in sers:
        if len(serlist)==len(rec.metadata):
            ser="all"
        else:
            serrange = [int(el.replace("Series","")) for el in serlist]
            if len(serrange)>1:
                ser = "Series%03i-%i"%(serrange[0],serrange[-1])
            else:
                ser = "Series%03i"%(serrange[0])
        outSer += [ser]
    return outSer

from copy import deepcopy

fracSaturTh = .05
movieFreq = 5
cmap = deepcopy(plt.cm.Greys)
cmap.set_bad("lime")

pd.set_option('display.max_rows', 100)

/opt/tljh/user/envs/physio/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.decomposition.incremental_pca module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.



### Preprocess (can take a minute or two)

In [3]:
recordings = []
for cur,ds,fs in os.walk(mainFolder):
    for f in fs:
#         if not (f.endswith(".lif") or f.endswith(".nd2")):
        if not f.endswith(".lif"):
            continue
        path = os.path.join(cur,f)
        month = int(os.path.split(cur)[1].split("_")[1])
        if month<3: continue
        recordings += [path]

recordings = sorted(recordings)

In [5]:
status = []
for pathToRecording in recordings:
    print (pathToRecording)
    rec = Recording(pathToRecording)
    recType = "Nikon" if pathToRecording.endswith(".nd2") else "Leica"

    if recType=="Leica":
        sers = parse_leica(rec)
    else:
        sers = ["all"]

    analysisFolder = os.path.join(rec.folder, rec.Experiment+"_analysis")
    if not os.path.isdir(analysisFolder):
        os.makedirs(analysisFolder)
        
    for ser in sers:
        saveDir = os.path.join(analysisFolder, ser)
        if not os.path.isdir(saveDir):
            os.makedirs(saveDir)
        rec.import_series(ser, onlyMeta=True)
        md = rec.Series[ser]["metadata"]
        md["exp"] = pathToRecording
        md["series"] = ser
        fs = get_filterSizes(md.pxSize)
#         md["filter sizes"] = fs
        movieFilename = os.path.join(saveDir, rec.Experiment+"_"+ser+".mp4")
        md["movie done"] = os.path.isfile(movieFilename)
        for k in ["bit depth", "Start time", "End time", "individual Series","Name"]:
            try:    del md[k]
            except: pass
        pklsDone = {}
        for fsize in fs:
            pickleFile = os.path.join(saveDir, ".".join(map(str,fsize))+"_rois.pkl")
            imageFile = os.path.join(saveDir, ".image_"+".".join(map(str,fsize))+".png")
            pickleThere = os.path.isfile(pickleFile)
            imageThere = os.path.isfile(imageFile)
            if pickleThere:
                purge = True
                try:
                    with open(pickleFile,"rb") as f:
                        rr = pickle.load(f)
                    purge = "trace" not in rr.df.columns
                except:
                    purge = True
                if purge:
                    os.remove(pickleFile)
                    if imageThere: os.remove(imageFile)
            pklsDone[fsize] = (pickleThere, os.path.isfile(imageFile))
        md["pickles"] = pklsDone
        md["allDone"] = np.array(list(pklsDone.values())).all()
        status += [dict(md.items())]
#     break

In [14]:
analysisFolder = os.path.join(rec.folder, rec.Experiment+"_analysis")

In [12]:
ser = "all"

In [6]:
rec = Recording(pathToRecording)

In [15]:
saveDir = os.path.join(analysisFolder, ser)

In [16]:
rec.import_series(ser, onlyMeta=True)

In [17]:
md = rec.Series[ser]["metadata"]
md["exp"] = pathToRecording
md["series"] = ser
fs = get_filterSizes(md.pxSize)
movieFilename = os.path.join(saveDir, rec.Experiment+"_"+ser+".mp4")
md["movie done"] = os.path.isfile(movieFilename)
for k in ["bit depth", "Start time", "End time", "individual Series","Name"]:
    try:    del md[k]
    except: pass

In [18]:
pklsDone = {}

In [19]:
for fsize in fs:
    break

In [20]:
pickleFile = os.path.join(saveDir, ".".join(map(str,fsize))+"_rois.pkl")
imageFile = os.path.join(saveDir, ".image_"+".".join(map(str,fsize))+".png")
pickleThere = os.path.isfile(pickleFile)
imageThere = os.path.isfile(imageFile)

In [21]:
pickleThere

True

In [22]:
with open(pickleFile,"rb") as f:
    rr = pickle.load(f)

In [ ]:
if pickleThere:
    purge = True
    try:
        with open(pickleFile,"rb") as f:
            rr = pickle.load(f)
        purge = "trace" not in rr.df.columns
    except:
        purge = True
    if purge:
        os.remove(pickleFile)
        if imageThere: os.remove(imageFile)

In [7]:
pklsDone[fsize] = (pickleThere, os.path.isfile(imageFile))
md["pickles"] = pklsDone
md["allDone"] = np.array(list(pklsDone.values())).all()
status += [dict(md.items())]

,Name,SizeT,SizeX,SizeY,pxSize,pxUnit,bit depth,Frequency,Start time,End time,Duration
0,Series009,32000,256,256,1.24,µm,uint8,50.1,2020-05-13 12:23:48,2020-05-13 12:34:27.255976810,00:10:39.255976
1,Series010,32000,256,256,1.24,µm,uint8,50.1,2020-05-13 12:34:27,2020-05-13 12:45:06.267978223,00:10:39.267978
2,Series011,32000,256,256,1.24,µm,uint8,50.1,2020-05-13 12:45:07,2020-05-13 12:55:46.280977210,00:10:39.280977
3,Series012,32000,256,256,1.24,µm,uint8,50.1,2020-05-13 12:55:47,2020-05-13 13:06:26.285976527,00:10:39.285976
4,Series013,32000,256,256,1.24,µm,uint8,50.1,2020-05-13 13:06:27,2020-05-13 13:17:06.290977752,00:10:39.290977
5,Series014,30275,256,256,1.24,µm,uint8,50.1,2020-05-13 13:17:07,2020-05-13 13:27:11.832977693,00:10:04.832977


In [10]:
rec.import_series("all", onlyMeta=True)

In [4]:
status = pd.DataFrame(status)
firstCols = ["exp", "series","allDone"]
status = status[firstCols+[c for c in status.columns if c not in firstCols]]
toDo = status[~status.allDone]

KeyError: "None of [Index(['exp', 'series', 'allDone'], dtype='object')] are in the [columns]"

In [ ]:
print (f"{len(status.exp.unique())} recording found.")
print (f"Out of them, {len(toDo.exp.unique())} need to be processed partially or fully.")

#### This is what will be processed:

In [6]:
for exp,df in toDo.groupby("exp"):
    print (f"from {exp}:" )
    display(df[["series", "SizeT", "SizeX", "SizeY", "Frequency","movie done"]])

from /data/Sandra/2020/2020_05_13/Experiment48b.lif:


,series,SizeT,SizeX,SizeY,Frequency,movie done
0,all,190275,256,256,50.1,True


from /data/Sandra/2020/2020_05_18/Experiment49a.lif:


,series,SizeT,SizeX,SizeY,Frequency,movie done
1,all,206324,128,128,84.2,True


from /data/Sandra/2020/2020_05_18/Experiment49b.lif:


...

,series,SizeT,SizeX,SizeY,Frequency,movie done
58,Series001,15546,128,128,19.8,False
59,Series002,21085,128,128,19.8,False
60,Series003,20605,128,128,19.8,False
61,Series004,30360,128,128,19.8,False
62,Series005,15046,128,128,19.8,False


from /data/Sandra/2020/2020_06_24/Experiment56d.lif:


,series,SizeT,SizeX,SizeY,Frequency,movie done
63,Series001,26126,256,256,19.800,False
64,Series002,24085,128,128,19.800,False
65,Series003,10852,128,128,19.800,False
66,Series004,26893,128,128,19.800,False
67,Series005,23582,128,128,19.800,False
68,Series006,6,512,512,0.734,False
69,Series007,12698,256,256,19.800,False
70,Series008,11630,256,256,19.800,False


from /data/Sandra/2020/2020_06_24/Experiment56e.lif:


,series,SizeT,SizeX,SizeY,Frequency,movie done
71,all,84579,128,128,19.8,False


### Finally, let's process them

In [7]:
for (pathToRecording, ser), df in toDo.groupby(["exp","series"]):
    print (pathToRecording,"\t", ser)
    rec = Recording(pathToRecording)
    toDrop = [i for i,row in rec.metadata.iterrows() if "Series" not in row["Name"]]
    rec.metadata.drop(index=toDrop, inplace=True)
    
    saveDir = os.path.join(rec.folder, rec.Experiment+"_analysis", ser)

    # import data
    try:
        rec.import_series(ser)
    except:
        print (f"could not import {ser} from {pathToRecording}")
        continue
    metadata = rec.Series[ser]["metadata"]
    movie = cmovie(rec.Series[ser]["data"],fr=metadata["Frequency"])
    # break
    # save movie
    movieFilename = os.path.join(saveDir, rec.Experiment+"_"+ser+".mp4")
    if not os.path.isfile(movieFilename):
        if movieFreq<movie.fr:
            showMovie = movie.resize(1,1,movieFreq/movie.fr)
            frameRate = 30
        else:
            showMovie = movie
            frameRate = 30*movie.fr/movieFreq
        show_movie(showMovie, 
               fps=frameRate,
               NTimeFrames=len(showMovie),
               out="save",
               saveName=movieFilename,
               tmax=len(movie)/movie.fr
              )
        del showMovie

    # anull saturated above threshold
    Nsatur = (movie==movie.max()).sum(0)
    toAnull = np.where(Nsatur>len(movie)*fracSaturTh)
    movie[(slice(None), )+toAnull] = 0

    # go thorugh filter sizes
    regions = Regions(movie, full=False, diag=True, debleach=False)
    filtSizes = get_filterSizes(metadata.pxSize)

    for spFilt in filtSizes:
        print ("\t"*2,"#"*5,spFilt)

        pickleFile = os.path.join(saveDir, ".".join(map(str,spFilt))+"_rois.pkl")
        if os.path.isfile(pickleFile) and os.path.getsize(pickleFile)>100:
            print ("already exists, skipping.")
            continue
        else:
            print ("processing with filter size of ", spFilt)
        regions.constructRois(gSig_filt=spFilt,img_th=0)
        regions.update(movie)
        while True:
            peak2bnd = getPeak2BoundaryDF(regions.df)
            df = peak2bnd.query("dist<1.")[["i","j"]]
            if len(df)==0: break
            gRois = getGraph_of_ROIs_to_Merge(df,regions,plot=False)
            dropped = mergeBasedOnGraph(gRois,regions)
            if dropped == 0: break

        regions.purge_lones((min(spFilt)*.4)**2)
        regions.sortFromCenter()
        saveRois(regions, saveDir, filename= ".".join(map(str,spFilt)), add_date=False, formats=["vienna"])

        fig = plt.figure(figsize=(5,5*np.divide(*movie.shape[1:])))
        ax = fig.add_axes([0.01,0.01,.98,.98])
        regions.plotEdges(imkw_args={"cmap":cmap},color="darkred", ax = ax)
        fig.savefig(os.path.join(saveDir, ".image_"+".".join(map(str,spFilt))+'.png'),dpi=150)
        plt.close(fig)

    del rec.Series[ser], regions, movie

/data/Sandra/2020/2020_05_13/Experiment48b.lif 	 all
		 ##### (5,)
processing with filter size of  (5,)
more than two attractors, not implemented yet, will skip
395 subsumed into existing ROIs.
12 subsumed into existing ROIs.
deleted 21 rois. 438 remain.
vienna
		 ##### (6,)
processing with filter size of  (6,)
245 subsumed into existing ROIs.
6 subsumed into existing ROIs.
deleted 10 rois. 303 remain.
vienna
		 ##### (5, 6)
processing with filter size of  (5, 6)
288 subsumed into existing ROIs.
6 subsumed into existing ROIs.
338 subsumed into existing ROIs.
3 subsumed into existing ROIs.
deleted 18 rois. 366 remain.
vienna
/data/Sandra/2020/2020_05_18/Experiment49a.lif 	 all
		 ##### (4,)
processing with filter size of  (4,)
92 subsumed into existing ROIs.
2 subsumed into existing ROIs.
deleted 3 rois. 144 remain.
vienna
		 ##### (5,)
processing with filter size of  (5,)
77 subsumed into existing ROIs.
1 subsumed into existing ROIs.
deleted 3 rois. 119 remain.
vienna
		 ##### (4, 5)
p

deleted 11 rois. 178 remain.
vienna
/data/Sandra/2020/2020_05_29/Experiment51a.lif 	 Series009
		 ##### (6,)
processing with filter size of  (6,)
196 subsumed into existing ROIs.
1 subsumed into existing ROIs.
deleted 10 rois. 243 remain.
vienna
		 ##### (7,)
processing with filter size of  (7,)
136 subsumed into existing ROIs.
1 subsumed into existing ROIs.
deleted 7 rois. 200 remain.
vienna
		 ##### (6, 7)
processing with filter size of  (6, 7)
122 subsumed into existing ROIs.
4 subsumed into existing ROIs.
deleted 10 rois. 220 remain.
vienna
		 ##### (6, 8)
processing with filter size of  (6, 8)
152 subsumed into existing ROIs.
5 subsumed into existing ROIs.
deleted 10 rois. 189 remain.
vienna
/data/Sandra/2020/2020_05_29/Experiment51a.lif 	 Series010
		 ##### (12,)
processing with filter size of  (12,)
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than t

more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
40 subsumed into existing ROIs.
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
1 subsumed into existing ROIs.
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
0 subsumed into existing ROIs.
deleted 1296 rois. 659 remain.
vienna
		 ##### (6, 8)
processing with filter size of  (6, 8)
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two a

more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
362 subsumed into existing ROIs.
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will ski

more than two attractors, not implemented yet, will skip
11656 subsumed into existing ROIs.
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will skip
more than two attractors, not implemented yet, will s

...

/home/jupyter-srdjan/srdjan_functs/islets/utils.py:118: RuntimeWarning:

divide by zero encountered in log

/opt/tljh/user/envs/physio/lib/python3.7/site-packages/matplotlib/image.py:399: RuntimeWarning:

invalid value encountered in double_scalars

/opt/tljh/user/envs/physio/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning:

invalid value encountered in less



		 ##### (4,)
processing with filter size of  (4,)
more than two attractors, not implemented yet, will skip
288 subsumed into existing ROIs.
more than two attractors, not implemented yet, will skip
3 subsumed into existing ROIs.
10 subsumed into existing ROIs.
1 subsumed into existing ROIs.
deleted 46 rois. 178 remain.
vienna
		 ##### (5,)
processing with filter size of  (5,)
244 subsumed into existing ROIs.
7 subsumed into existing ROIs.
deleted 39 rois. 135 remain.
vienna
		 ##### (4, 5)
processing with filter size of  (4, 5)
217 subsumed into existing ROIs.
7 subsumed into existing ROIs.
deleted 23 rois. 163 remain.
vienna
		 ##### (4, 6)
processing with filter size of  (4, 6)
311 subsumed into existing ROIs.
9 subsumed into existing ROIs.
1 subsumed into existing ROIs.
deleted 31 rois. 137 remain.
vienna
/data/Sandra/2020/2020_06_18/Experiment54d.lif 	 Series004


/home/jupyter-srdjan/srdjan_functs/islets/utils.py:118: RuntimeWarning:

divide by zero encountered in log

/opt/tljh/user/envs/physio/lib/python3.7/site-packages/matplotlib/image.py:399: RuntimeWarning:

invalid value encountered in double_scalars

/opt/tljh/user/envs/physio/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning:

invalid value encountered in less



		 ##### (4,)
processing with filter size of  (4,)
225 subsumed into existing ROIs.
5 subsumed into existing ROIs.
deleted 51 rois. 155 remain.
vienna
		 ##### (5,)
processing with filter size of  (5,)
196 subsumed into existing ROIs.
4 subsumed into existing ROIs.
deleted 30 rois. 128 remain.
vienna
		 ##### (4, 5)
processing with filter size of  (4, 5)
156 subsumed into existing ROIs.
3 subsumed into existing ROIs.
deleted 36 rois. 143 remain.
vienna
		 ##### (4, 6)
processing with filter size of  (4, 6)


OSError: [Errno 12] Cannot allocate memory

In [11]:
with open(pickleFile.replace("5.7","5"),"rb") as f:
    rr = pickle.load(f)

In [12]:
rr.df.columns

Index(['peak', 'pixels'], dtype='object')